# IPL Chatbot

Original: Learn how to Build and Deploy a Chatbot in Minutes using Rasa (IPL Case Study!) - Mohd Sanad Zaki Rizvi, Analytics Vidhya https://www.analyticsvidhya.com/blog/2019/04/learn-build-chatbot-rasa-nlp-ipl)

**Objectives** To build a chatbot capable of fetching latest info about the ongoing IPL (Indian Premier League) matches from cricapi.com site.

<img src="images/ipl.jpg">

### Importing Libraries

In [17]:
%matplotlib inline

# First things first
import nest_asyncio
nest_asyncio.apply()
print("Event loop ready.")

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

import rasa_nlu
import rasa_core
import spacy

Event loop ready.


### Preparing the NLU Training Data

Training data for extracting the user intent.
As you can see, the format of training data for ‘intent’ is quite simple in Rasa. You just have to:

- Start the line with “## intent:intent_name”
- Supply all the examples in the following lines

In [18]:
nlu_md = """
## intent:goodbye  
- Bye 
- Goodbye
- See you later
- Bye bot
- Goodbye friend
- bye
- bye for now
- catch you later
- gotta go
- See you
- goodnight
- have a nice day
- i'm off
- see you later alligator
- we'll speak soon
- end
- finish

## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks
- hi Mister
- hi pal!
- hi there
- greetings
- hello everybody
- hello is anybody there
- hello robot
- who are you?
- what are you?
- what's up
- how do you do?

## intent:thanks
- Thanks
- Thank you
- Thank you so much
- Thanks bot
- Thanks for that
- cheers
- cheers bro
- ok thanks!
- perfect thank you
- thanks a bunch for everything
- thanks for the help
- thanks a lot
- amazing, thanks
- cool, thanks
- cool thank you

## intent:affirm
- y
- Y
- yes
- yes sure
- absolutely
- for sure
- yes yes yes
- definitely
- yes, it did.

## intent:current_matches
- what are the current matches
- can you list the matches in ipl 2019
- which cricket match is happening right now
- which ipl match is next
- which teams are playing next in ipl
- which team will play next in ipl
- tell me some ipl news
- i want ipl updates
- can you give me ipl latest updates
- what are the latest match updates
- who won the last ipl match
- which teams are competing in the next match
- how is ipl going
- what was the result of the last match
- when is the next match

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really
- n
- N
"""

%store nlu_md > data/ipl_nlu_data.md

Writing 'nlu_md' (str) to file 'data/ipl_nlu_data.md'.


You can include as many examples as you want for each intent. In fact, make sure to include slangs and short forms that you use while texting. The idea is to make the chatbot understand the way we type text. Feel free to refer to the complete version where I have given plenty of examples for each intent type.

### Defining the NLU Model Configuration

This file lets us create a text processing pipeline in Rasa. Luckily for us, Rasa comes with two default settings based on the amount of training data we have:
- “spacy_sklearn” pipeline if you have less than 1000 training examples
- “tensorflow_embedding” if you have a large amount of data

In [19]:
config = """
language: "en"
pipeline: spacy_sklearn
""" 

%store config > config/ipl_nlu_config.yml

Writing 'config' (str) to file 'config/ipl_nlu_config.yml'.


### Training the NLU Classifier Model

On command line you can run following command:

**python -m rasa_nlu.train -c ipl_nlu_config.yml --data data/ipl_nlu_data.md -o models --fixed_model_name nlu --project current --verbose**

Or programmatically you can write code

In [20]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("data/ipl_nlu_data.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config/ipl_nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/ipl_nlu", fixed_model_name="current")

INFO:rasa.nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa_nlu.model:Starting to train component SpacyNLP
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyTokenizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component RegexFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SklearnIntentClassifier


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into 'C:\YogeshKulkarni\Training\TeachingDataScience\Jupyter\models\ipl_nlu\current'


### Evaluating the NLU model on a random text (first way)

Let’s test how good our model is performing by giving it a sample text that it hasn’t been trained on for extracting intent. 

In [33]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("what is happening in the cricket world these days?"))

{
  "intent": {
    "name": "current_matches",
    "confidence": 0.517677511039713
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "current_matches",
      "confidence": 0.517677511039713
    },
    {
      "name": "greet",
      "confidence": 0.25656597604201004
    },
    {
      "name": "goodbye",
      "confidence": 0.09646234821419979
    },
    {
      "name": "affirm",
      "confidence": 0.06219961483646996
    },
    {
      "name": "deny",
      "confidence": 0.035063641571578326
    },
    {
      "name": "thanks",
      "confidence": 0.032030908296028435
    }
  ],
  "text": "what is happening in the cricket world these days?"
}


Not only does our NLU model perform well on intent extraction, but it also ranks the other intents based on their confidence scores. This is a nifty little feature that can be really useful when the classifier is confused between multiple intents.

### Evaluating the NLU model on a random text (2nd way)
Let’s test how good our model is performing by giving it a sample text that it hasn’t been trained on for extracting intent. You can open an iPython/Python shell and follow the following steps:

In [34]:
from rasa_nlu.model import Interpreter
nlu_model = Interpreter.load('./models/ipl_nlu/current')
nlu_model.parse('what is happening in the cricket world these days?')

INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.


{'intent': {'name': 'current_matches', 'confidence': 0.517677511039713},
 'entities': [],
 'intent_ranking': [{'name': 'current_matches',
   'confidence': 0.517677511039713},
  {'name': 'greet', 'confidence': 0.25656597604201004},
  {'name': 'goodbye', 'confidence': 0.09646234821419979},
  {'name': 'affirm', 'confidence': 0.06219961483646996},
  {'name': 'deny', 'confidence': 0.035063641571578326},
  {'name': 'thanks', 'confidence': 0.032030908296028435}],
 'text': 'what is happening in the cricket world these days?'}

### Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [35]:
from rasa_nlu.test  import run_evaluation

run_evaluation("data/ipl_nlu_data.md", model_directory)

INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa_nlu.test:Running model for predictions:
100%|█████████████████████████████████████████████████████████████████████████████████| 83/83 [00:00<00:00, 254.08it/s]
INFO:rasa_nlu.test:Intent evaluation results:
INFO:rasa_nlu.test:Intent Evaluation: Only considering those 83 examples that have a defined intent out of 83 examples
INFO:rasa_nlu.test:F1-Score:  1.0
INFO:rasa_nlu.test:Precision: 1.0
INFO:rasa_nlu.test:Accuracy:  1.0
INFO:rasa_nlu.test:Classification report: 
                 precision    recall  f1-score   support

current_matches       1.00      1.00      1.00        15
         thanks       1.00      1.00      1.00        15
         affirm       1.00      1.00      1.00         9
          greet       1.00      1.00      1.00        19
           deny       1.00      1.00      1.00         8
        goodbye       1.00      1.00      1.00        17

      micro avg       1.00      1.00  

{'intent_evaluation': {'predictions': [{'text': 'Bye',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.9678515873146009},
   {'text': 'Goodbye',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.8876912614811194},
   {'text': 'See you later',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.7977705943125113},
   {'text': 'Bye bot',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.8186983759659981},
   {'text': 'Goodbye friend',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.8325974252404662},
   {'text': 'bye',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.9678515873146009},
   {'text': 'bye for now',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.771397988783037},
   {'text': 'catch you later',
    'intent': 'goodbye',
    'predicted': 'goodbye',
    'confidence': 0.7673529098380265},
   {'text': 'gotta go',
    

## Rasa Core: Making Interactive Conversations

One of the most important aspects of a chatbot application is its ability to be interactive. 

### Designing the conversational flow

Think of the simplest conversation our chatbot can have with a user. What would be the flow of such a conversation?

---
Me: Hi

Iplbot: Hey! How may I help you?

Me: What was the result of the last match?

Iplbot: Here are some IPL quick info:
1.The match between Rajasthan Royals and Delhi Capitals was recently held and Delhi Capitals won.
2.The next match is Warriors vs Titans on 22 April 2019

Iplbot: Did that help you?

Me: yes, thank you!

Iplbot: Glad that I could help!

---
Let’s see how we can teach a simple conversation like that to Rasa:


The general format is:

This is called a user story path. I have provided a few stories in the data/stories.md file for your reference. This is the training data for Rasa Core.

### Writing  Stories

The way it works is:

- Give some examples of sample story paths that the user is expected to follow
- Rasa Core combines them randomly to create more complex user paths
- It then builds a probabilistic model out of that. This model is used to predict the next action Rasa should take

<img src="images/conversation_flow.png">

The above illustration might look complicated, but it’s simply listing out various possible user stories that I have taught Rasa. Here are a few things to note from the above graph:

- Except for the START and END boxes, all the colored boxes indicate user intent
- All the white boxes are actions that the chatbot performs
- Arrows indicate the flow of the conversation
- action_match_news is where we hit the CricAPI to get IPL information

In [36]:
stories_md = """
## news path 1
* greet
  - utter_greet
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## news path 2
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## news path 3
* greet
  - utter_greet
* current_matches
  - action_match_news
  - utter_did_that_help
* deny
  - utter_ask_again
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## greet path
* greet
  - utter_greet

## goodbye path
* goodbye
  - utter_goodbye
"""

%store stories_md > data/ipl_stories.md

Writing 'stories_md' (str) to file 'data/ipl_stories.md'.


Now, generate a similar graph for your stories using the following command:

**python -m rasa_core.visualize -d domain.yml -s data/ipl_stories.md -o graph.html**

This is very helpful when debugging the conversational flow of the chatbot.

### Defining the Domain

The domain is the world of your chatbot. It contains everything the chatbot should know, including:

- All the actions it is capable of doing
- The intents it should understand
- The template of all the utterances it should tell the user, and much more

In [37]:
domain_yml = """
actions:
- utter_greet
- utter_did_that_help
- utter_goodbye
- action_match_news
- utter_default
- utter_gratitude
- utter_ask_again

intents:
- goodbye
- greet
- thanks
- current_matches
- affirm
- deny

templates:
  utter_greet:
  - text: "Hey! What can I do for you?"
  utter_did_that_help:
  - text: "Did that help you?"
  - text: "I hope that solved your query"
  utter_goodbye:
  - text: "Bye"
  utter_default:
  - text: "I am sorry, I didn't get that. Could you please repeat your query?"
  - text: "I am not sure what you are aiming for."
  utter_gratitude:
  - text: "Glad that I could be of help to you!\nBye"
  utter_ask_again:
  - text: "Okay! Let's start again, please tell me what do you need?"
  - text: "No issues! Let's try this again.\n Please repeat your query?"
"""

%store domain_yml > config/ipl_domain.yml

Writing 'domain_yml' (str) to file 'config/ipl_domain.yml'.


### Setting Policies

Rasa Core generates the training data for the conversational part using the stories we provide. It also lets you define a set of policies to use when deciding the next action of the chatbot. These policies are defined in the policies.yml file.

In [38]:
policies_yml = """
policies:
  - name: KerasPolicy
    epochs: 100
    max_history: 5
  - name: FallbackPolicy
    fallback_action_name: 'action_default_fallback'
  - name: MemoizationPolicy
    max_history: 5
"""

%store policies_yml > config/ipl_policies.yml

Writing 'policies_yml' (str) to file 'config/ipl_policies.yml'.


Notes:
- KerasPolicy uses a neural network implemented in Keras to select the next action. The default architecture is based on an LSTM (Long Short Term Memory) model
- MemoizationPolicy memorizes the conversations in your training data. It predicts the next action with confidence 1.0 if this exact conversation exists in the training data, otherwise, it predicts ‘None’ with confidence 0.0
- FallbackPolicy invokes a fallback action if the intent recognition has confidence below nlu_threshold or if none of the dialogue policies predict action with confidence higher than core_threshold
- One important hyperparameter for Rasa Core policies is the max_history. This controls how much dialogue history the model looks at to decide which action to take next

###  Custom Actions

Using CricAPI for fetching IPL related news. It is free for 100 requests per day, which (I hope) is more than enough to satiate that cricket crazy passion you have.

You need to first signup on the website to get access to their API:
https://www.cricapi.com/

You should be able to see your API Key once you are logged in:

<img src="images/lala-1140x399.png">

Modifications to original code:

- Instead of showing API key here it has been stored in ENV variable and fetched here
- Key "toss_winner_team" was subsituted into depreceated key

In [39]:
actions_py="""
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from datetime import datetime

import logging
import requests
import json
import os
from rasa_sdk import Action

logger = logging.getLogger(__name__)

API_URL = "https://cricapi.com/api/"
API_KEY = os.environ.get('CRICINFOAPI')

class ApiAction(Action):
    def name(self):
        return "action_match_news"

    def run(self, dispatcher, tracker, domain):
        print(API_URL + "matches" + "?apikey=" + API_KEY)
        res = requests.get(API_URL + "matches" + "?apikey=" + API_KEY) #, verify=False
        if res.status_code == 200:
            data = res.json()["matches"]
            recent_match = data[0]
            upcoming_match = data[1]
            upcoming_match["date"] = datetime.strptime(upcoming_match["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
            next_date = upcoming_match["date"].strftime("%d %B %Y")

            out_message = "Here some IPL quick info: 1.The match between {} and {} was recently held and {} won the toss.".format(recent_match["team-1"], recent_match["team-2"], recent_match["toss_winner_team"])

            dispatcher.utter_message(out_message)

            out_message = "2.The next match is {} vs {} on {}".format(upcoming_match["team-1"], upcoming_match["team-2"], next_date)

            dispatcher.utter_message(out_message)

            return []
"""
%store actions_py > ipl_actions.py

Writing 'actions_py' (str) to file 'ipl_actions.py'.


Need endpoints yml to execute the actions server.

Note: If you have external API call, like REST, need to have "webhook" word at the end, else nothing.

My own query on this topic: https://forum.rasa.com/t/rasa-core-sdk-not-working/9228

In [40]:
endpoints_yml = """
#action_endpoint:
#  url: "http://localhost:5055/webhook"
  
action_endpoint:
  url: http://localhost:5055/webhook

#nlg:
#  url: http://localhost:5056/nlg

core_endpoint:
  url: http://localhost:5005
"""
%store endpoints_yml > endpoints.yml

Writing 'endpoints_yml' (str) to file 'endpoints.yml'.


In a separate shell (cmd for Windows):
- **activate rasa**
- come to directory where actions.py is and then run
- **python -m rasa_sdk.endpoint --actions ipl_actions**

This way, custom action server starts ...

In [41]:
# import sys
# python = sys.executable
# !{python} -m rasa_core_sdk.endpoint --actions actions

###  Visualising the Training Data

### Training a Dialogue Model

You can train the model using the following command:

**python -m rasa_core.train -d ipl_domain.yml -s data/ipl_stories.md -o models/current/dialogue -c ipl_policies.yml**

Or do it programmatically as:

In [ ]:
import asyncio
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy, MappingPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

# agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

agent = Agent('config/ipl_domain.yml', policies = [MemoizationPolicy(max_history=2), KerasPolicy(validation_split=0.2,epochs=200),MappingPolicy()])
# loading our neatly defined training dialogues
training_data = await agent.load_data('data/ipl_stories.md')

agent.train(training_data)
# FOLLOWING WAY OF giving arguments is depreacted, instead, give these params as arguments to KerasPolicy
# agent.train(
#     training_data,
#     validation_split=0.0,
#     epochs=200
# )

agent.persist('models/ipl_dialogue/current')

Processed trackers: 100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 116.59it/s, # actions=15]
Processed actions: 15it [00:00, 940.06it/s, # examples=15]
Processed trackers: 100%|██████████████████████████████████████████████| 121/121 [00:01<00:00, 65.47it/s, # actions=64]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 21)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6912      
_________________________________________________________________
dense (Dense)                (None, 15)                495       
_________________________________________________________________
activation (Activation)      (None, 15)                0         
Total params: 7,407
Trainable params: 7,407
Non-trainable params: 0
_________________________________________________________________


INFO:rasa.core.policies.keras_policy:Fitting model with 64 total samples and a validation split of 0.2


Epoch 1/200
64/64 [==============================] - ETA: 0s - loss: 2.6763 - acc: 0.156 - 0s 7ms/sample - loss: 2.6860 - acc: 0.0781
Epoch 2/200
64/64 [==============================] - ETA: 0s - loss: 2.6435 - acc: 0.187 - 0s 249us/sample - loss: 2.6514 - acc: 0.1250
Epoch 3/200
64/64 [==============================] - ETA: 0s - loss: 2.6181 - acc: 0.218 - 0s 187us/sample - loss: 2.6200 - acc: 0.1250
Epoch 4/200
64/64 [==============================] - ETA: 0s - loss: 2.6004 - acc: 0.250 - 0s 234us/sample - loss: 2.5994 - acc: 0.1406
Epoch 5/200
64/64 [==============================] - ETA: 0s - loss: 2.5575 - acc: 0.281 - 0s 265us/sample - loss: 2.5659 - acc: 0.1875
Epoch 6/200
64/64 [==============================] - ETA: 0s - loss: 2.5363 - acc: 0.125 - 0s 249us/sample - loss: 2.5397 - acc: 0.1719
Epoch 7/200
64/64 [==============================] - ETA: 0s - loss: 2.5168 - acc: 0.312 - 0s 249us/sample - loss: 2.5160 - acc: 0.2500
Epoch 8/200
64/64 [==============================]

Epoch 61/200
64/64 [==============================] - ETA: 0s - loss: 1.6338 - acc: 0.312 - 0s 234us/sample - loss: 1.5736 - acc: 0.4062
Epoch 62/200
64/64 [==============================] - ETA: 0s - loss: 1.5997 - acc: 0.343 - 0s 218us/sample - loss: 1.5591 - acc: 0.4375
Epoch 63/200
64/64 [==============================] - ETA: 0s - loss: 1.6133 - acc: 0.375 - 0s 218us/sample - loss: 1.5715 - acc: 0.4219
Epoch 64/200
64/64 [==============================] - ETA: 0s - loss: 1.5620 - acc: 0.343 - 0s 171us/sample - loss: 1.5119 - acc: 0.4375
Epoch 65/200
64/64 [==============================] - ETA: 0s - loss: 1.5882 - acc: 0.343 - 0s 172us/sample - loss: 1.5423 - acc: 0.4375
Epoch 66/200
64/64 [==============================] - ETA: 0s - loss: 1.5399 - acc: 0.343 - 0s 171us/sample - loss: 1.5190 - acc: 0.4531
Epoch 67/200
64/64 [==============================] - ETA: 0s - loss: 1.5628 - acc: 0.375 - 0s 171us/sample - loss: 1.5111 - acc: 0.4688
Epoch 68/200
64/64 [=====================

64/64 [==============================] - ETA: 0s - loss: 1.1598 - acc: 0.687 - 0s 203us/sample - loss: 1.0817 - acc: 0.7344
Epoch 121/200
64/64 [==============================] - ETA: 0s - loss: 1.0608 - acc: 0.625 - 0s 171us/sample - loss: 1.0401 - acc: 0.6562
Epoch 122/200
64/64 [==============================] - ETA: 0s - loss: 1.1009 - acc: 0.625 - 0s 187us/sample - loss: 1.0433 - acc: 0.6875
Epoch 123/200
64/64 [==============================] - ETA: 0s - loss: 1.0040 - acc: 0.687 - 0s 171us/sample - loss: 1.0156 - acc: 0.7344
Epoch 124/200
64/64 [==============================] - ETA: 0s - loss: 1.1041 - acc: 0.687 - 0s 171us/sample - loss: 1.0753 - acc: 0.6719
Epoch 125/200
64/64 [==============================] - ETA: 0s - loss: 1.0034 - acc: 0.750 - 0s 171us/sample - loss: 0.9951 - acc: 0.7656
Epoch 126/200
64/64 [==============================] - ETA: 0s - loss: 1.0917 - acc: 0.625 - 0s 171us/sample - loss: 1.0371 - acc: 0.7344
Epoch 127/200
64/64 [===========================

64/64 [==============================] - ETA: 0s - loss: 0.6701 - acc: 0.937 - 0s 203us/sample - loss: 0.6273 - acc: 0.9219
Epoch 180/200
64/64 [==============================] - ETA: 0s - loss: 0.5924 - acc: 0.968 - 0s 187us/sample - loss: 0.5593 - acc: 0.9688
Epoch 181/200
64/64 [==============================] - ETA: 0s - loss: 0.6149 - acc: 1.000 - 0s 187us/sample - loss: 0.5851 - acc: 0.9688
Epoch 182/200
64/64 [==============================] - ETA: 0s - loss: 0.5701 - acc: 1.000 - 0s 171us/sample - loss: 0.5609 - acc: 0.9844
Epoch 183/200
64/64 [==============================] - ETA: 0s - loss: 0.5437 - acc: 1.000 - 0s 171us/sample - loss: 0.5607 - acc: 0.9688
Epoch 184/200
64/64 [==============================] - ETA: 0s - loss: 0.5969 - acc: 0.968 - 0s 187us/sample - loss: 0.5681 - acc: 0.9688
Epoch 185/200
64/64 [==============================] - ETA: 0s - loss: 0.5112 - acc: 1.000 - 0s 171us/sample - loss: 0.5273 - acc: 0.9688
Epoch 186/200
64/64 [===========================

INFO:rasa.core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Persisted model to 'C:\YogeshKulkarni\Training\TeachingDataScience\Jupyter\models\ipl_dialogue\current\core'


## Running: Talk to your Bot

So we have the chatbot ready. It’s time to chat

One way is to run following command in shell (windows cmd)
- activate the environment,
- come to directory where actions.py is and then run
- **python -m rasa_core.run -d models/current/ipl_dialogue -u models/current/ipl_nlu --endpoints ipl_endpoints.yml**

Or else do programmatically like below

Both approaches expect rasa core sdk server running in a separate window, else **python -m rasa_core_sdk.endpoint --actions ipl_actions --endpoints ipl_endpoints.yml**

In [2]:
# #Starting the Bot

# from rasa_core.agent import Agent
# agent = Agent.load('models/current/dialogue', interpreter=model_directory)

import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa.utils.endpoints import EndpointConfig
import asyncio
import time

def load_assistant():
    messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
    model_dir = 'models/ipl_nlu/current' # or model_directory defined earlier
    interpreter = NaturalLanguageInterpreter.create(model_dir)
    endpoint = EndpointConfig('http://localhost:5055/webhook')
    agent = Agent.load('models/ipl_dialogue/current', interpreter=interpreter, action_endpoint = endpoint)

    print("Your bot is ready to talk! Type your messages here or send 'stop'")
    while True:
        a = input()
        if a == 'stop':
            break
        responses = asyncio.run(agent.handle_text(a)) # For Python 3.7
        for response in responses:
            print(response["text"])

C:\Users\yogesh.kulkarni\AppData\Local\Continuum\anaconda3\envs\rasa\lib\site-packages\rasa_core\__init__.py:12: UserWarning: The 'rasa_core' package has been renamed. You should change your imports to use 'rasa.core' instead.
  UserWarning,
C:\Users\yogesh.kulkarni\AppData\Local\Continuum\anaconda3\envs\rasa\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\yogesh.kulkarni\AppData\Local\Continuum\anaconda3\envs\rasa\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\yogesh.kulkarni\AppData\Local\Continuum\anaconda3\envs\rasa\lib\si

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.









In [3]:
load_assistant()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Your bot is ready to talk! Type your messages here or send 'stop'
Hi


RuntimeError: asyncio.run() cannot be called from a running event loop

## What Next?

- Try to use different pipelines in Rasa Core, explore more Policies, fine-tune those models, 
- Check out what other features CricAPI provides, etc.
- Other APIs
- Slot filling
- Different languages (Hindi bot?)

